# CKG

This notebook explores the biomedical knowledge graph provided by the project **Clinical Knowledge Graph (CKG)**: [Publication (2023)](https://doi.org/10.1038/s41587-021-01145-6), [Website](https://ckg.readthedocs.io), [Code](https://github.com/MannLabs/CKG), [Data](https://doi.org/10.17632/mrcf7f4tc2)

The source file of this notebook is [ckg.ipynb](https://github.com/robert-haas/awesome-biomedical-knowledge-graphs/blob/main/src/notebooks/ckg.ipynb) and can be found in the repository [awesome-biomedical-knowledge-graphs](https://github.com/robert-haas/awesome-biomedical-knowledge-graphs) that also contains information about similar projects.


## Table of contents

1. [Setup](#1.-Setup)
2. [Data download](#2.-Data-download)
3. [Data extraction](#2.-Data-extraction)
4. [Data import](#3.-Data-import)
5. [Data inspection](#4.-Data-inspection)
6. [Schema discovery](#5.-Schema-discovery)
7. [Knowledge graph reconstruction](#6.-Knowledge-graph-reconstruction)
8. [Subgraph exploration](#7.-Subgraph-exploration)

## 1. Setup

This section prepares the environment for the following exploratory data analysis.

### a) Import packages

From the [Python standard library](https://docs.python.org/3/library/index.html).

In [ ]:
import json
import os

From the [Python Package Index (PyPI)](https://pypi.org).

In [ ]:
import dask.dataframe as dd
import gravis as gv
import igraph as ig

In [ ]:
# A docker installation is required to load and convert the Neo4j database inside a suitable container
import docker

From a local Python module named [shared_bmkg.py](https://github.com/robert-haas/awesome-biomedical-knowledge-graphs/blob/main/src/notebooks/shared_bmkg.py). The functions in it are used in several similar notebooks to reduce code repetition and to improve readability.

In [ ]:
import shared_bmkg

### b) Create data directories

The raw data provided by the project and the transformed data generated throughout this notebook are stored in separate directories. If the notebook is run more than once, the downloaded data is reused instead of fetching it again, but all data transformations are rerun.

In [ ]:
project_name = "ckg"
download_dir = os.path.join(project_name, "downloads")
results_dir = os.path.join(project_name, "results")

shared_bmkg.create_dir(download_dir)
shared_bmkg.create_dir(results_dir)

## 2. Data download

This section fetches the data published by the project on [Mendeley Data](https://data.mendeley.com/datasets/mrcf7f4tc2/3). The latest available version at the time of creating this notebook was used: `Version 3 (2021-08-17)`.

### All files provided by the project

- `ckg_latest_4.2.3.dump`: Neo4j graph database with nodes, edges and annotations.
- `data.zip`: Data about experimental studies presented in the publication.

### Files needed to create the knowledge graph

- `ckg_latest_4.2.3.dump` contains all information required for reconstructing the knowledge graph.

In [ ]:
download_specification = [
    ("ckg_latest_4.2.3.dump", "https://data.mendeley.com/public-files/datasets/mrcf7f4tc2/files/ffaab45e-e15c-412d-b63b-5df681a2e303/file_downloaded", "eebe895e2e9f9fc39f3663fbcea032d5"),
    ("data.zip", "https://data.mendeley.com/public-files/datasets/mrcf7f4tc2/files/69de0ef6-6e71-4d8e-8fbc-b933b9fc4dce/file_downloaded", "1bbd8be31efcd559244b26f474d9ad59"),

    # APOC library for Neo4j to enable exports of the knowledge graph in CSV format
    ("apoc-4.4.0.24-all.jar", "https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/download/4.4.0.24/apoc-4.4.0.24-all.jar", "7c6a702322b0aaf663c25f378cd3494d"),
]

for filename, url, md5 in download_specification:
    filepath = os.path.join(download_dir, filename)
    shared_bmkg.fetch_file(url, filepath)
    shared_bmkg.validate_file(filepath, md5)
    print()

## 3. Data extraction

This section extracts the raw data from the Neo4j database into two CSV files. It uses a Docker container with a specific Neo4j+APOC installation and parameterization that allows to load and export the database dump file. It was tricky to figure out a configuration that works with this specific dump file.

In [ ]:
def check_docker_installation():
    try:
        client = docker.from_env()
        
        # Pull the hello-world image
        client.images.pull('hello-world')
        
        # Run the hello-world container
        container = client.containers.run('hello-world', detach=True)
        output = container.logs()
        
        # Check the output for the expected message
        if b"Hello from Docker!" in output:
            print("Docker is installed and running properly. The hello-world container output is correct.")
            success = True
        else:
            print("Docker is installed but the hello-world container output is incorrect.")
            success = False
        
        # Clean-up: Remove the container and image
        container.remove()
        client.images.remove('hello-world')
        return success
    except (DockerException, ImageNotFound, ContainerError) as e:
        print(f"Docker is not installed or not running properly: {e}")
        return False


check_docker_installation()

In [ ]:
def run_docker_command(container, command, verbose):
    # Report the command
    if verbose:
        print('Running command:', command)

    # Run the command
    exit_code, output = container.exec_run(command)

    # Report the result
    if verbose:
        print(' Worked!' if exit_code == 0 else f'Failed! Exit code: {exit_code}')
        try:
            output = output.decode()
            output = output.strip()
        except Exception:
            pass
        if output:
            print(' Output:', output)
        print()


def convert_neo4j_dump_to_csv_files(download_dir, results_dir, filename_db, filename_apoc, verbose=False):
    # Install APOC and load the export functions: https://neo4j.com/docs/apoc/5/installation
    # Use the export functions: https://neo4j.com/labs/apoc/4.4/overview/apoc.export

    # Construct source filepaths
    filepath_db_src = os.path.join(download_dir, filename_db)
    filepath_apoc_src = os.path.join(download_dir, filename_apoc)

    # Create a temporary destination directory
    dirname_dst = "tempdir_for_docker"
    dirpath_dst = os.path.abspath(os.path.join(download_dir, dirname_dst))
    os.makedirs(dirpath_dst, exist_ok=True)

    # Create the destination filepaths
    filepath_db_dst = os.path.join(dirpath_dst, "neo4j.dump")
    filepath_apoc_dst = os.path.join(dirpath_dst, filename_apoc)

    # Move files from source to destination directory
    os.rename(filepath_db_src, filepath_db_dst)
    os.rename(filepath_apoc_src, filepath_apoc_dst)

    # Fetch and run the Neo4j v4.4 Docker container
    client = docker.from_env()
    if verbose:
        print('Starting Docker container:', end=' ')
    container = client.containers.run(
        "neo4j:4.4",
        detach=True,
        volumes={dirpath_dst: {'bind': '/mnt', 'mode': 'rw'}},
        working_dir='/mnt',
        entrypoint='tail -f /dev/null',
    )
    if verbose:
        print(container)
        print()

    # Convert the .dump file to a .csv file
    try:
        commands = [
            # Configure Neo4j for the import
            '''sh -c "echo 'dbms.allow_upgrade=true' >> /var/lib/neo4j/conf/neo4j.conf"''',
            '''sh -c "echo 'dbms.security.procedures.allowlist=apoc.*' >> /var/lib/neo4j/conf/neo4j.conf"''',
            '''sh -c "echo 'dbms.security.procedures.unrestricted=apoc.*' >> /var/lib/neo4j/conf/neo4j.conf"''',
            '''sh -c "echo 'apoc.export.file.enabled=true' >> /var/lib/neo4j/conf/neo4j.conf"''',
            '''sh -c "neo4j-admin set-initial-password correcthorsebatterystaple"''',
            # Make APOC extension available to Neo4j
            f'''sh -c "cp /mnt/{filename_apoc} /var/lib/neo4j/plugins"''',
            # Import the .dump file
            f'''sh -c "neo4j-admin load --from=/mnt/neo4j.dump --database=neo4j --force"''',
            # Start the database
            '''sh -c "rm -rf /var/lib/neo4j/logs && neo4j start"''',
            # Wait so it is ready
            '''sh -c "sleep 60"''',
            # Export a .csv file for nodes
            '''cypher-shell -u neo4j -p correcthorsebatterystaple -d neo4j "CALL apoc.export.csv.query('MATCH (n) RETURN id(n) as id, labels(n)[0] as type, properties(n) as properties', 'nodes.csv', {})"''',
            # Export a .csv file for edges
            '''cypher-shell -u neo4j -p correcthorsebatterystaple -d neo4j "CALL apoc.export.csv.query('MATCH ()-[r]->() RETURN id(startNode(r)) as source_id, id(endNode(r)) as target_id, type(r) as type, properties(r) as properties', 'edges.csv', {})"''',
            # Move the .csv files to the mounted directory to make them available outside the container
            '''sh -c "mv /var/lib/neo4j/import/nodes.csv /mnt"''',
            '''sh -c "mv /var/lib/neo4j/import/edges.csv /mnt"''',
        ]
        for cmd in commands:
            run_docker_command(container, cmd, verbose)
    except Exception as e:
        print(e)
    finally:
        if container:
            if verbose:
                print('Stopping and removing the container')
            container.stop()
            container.remove()

        # Move the original files from temporary destination to source directory
        os.rename(filepath_db_dst, filepath_db_src)
        os.rename(filepath_apoc_dst, filepath_apoc_src)

        # Move the new .csv file from temporary destination to results directory
        os.rename(os.path.join(dirpath_dst, "nodes.csv"), os.path.join(results_dir, "nodes.csv"))
        os.rename(os.path.join(dirpath_dst, "edges.csv"), os.path.join(results_dir, "edges.csv"))

        # Delete the temporary destination directory
        os.rmdir(dirpath_dst)
    return container

In [ ]:
%%time

filepath_nodes = os.path.join(results_dir, "nodes.csv")
filepath_edges = os.path.join(results_dir, "edges.csv")

if not (os.path.exists(filepath_nodes) and os.path.exists(filepath_edges)):
    print(f"Starting the conversion of the .dump file to two .csv files.")
    # takes about 45min to create nodes.csv and edges.csv
    container = convert_neo4j_dump_to_csv_files(
        download_dir,
        results_dir,
        "ckg_latest_4.2.3.dump",
        "apoc-4.4.0.24-all.jar",
        verbose=True,
    )
else:
    print(f"Found existing files {filepath_nodes} and {filepath_edges}. Using these instead of performing another conversion.")

## 4. Data import

This section loads the extracted data files that are required for reconstructing the knowledge graph.

In [ ]:
def read_csv_file(filepath):
    with open(filepath) as f:
        # A Dask dataframe, not Pandas
        df = dd.read_csv(filepath, dtype=str)
    return df

In [ ]:
%%time

df_nodes = read_csv_file(os.path.join(results_dir, "nodes.csv"))
df_edges = read_csv_file(os.path.join(results_dir, "edges.csv"))

## 5. Data inspection

This section attempts to reproduce some published numbers by inspecting the raw data and then prints a few exemplary records.

The [publication](https://doi.org/10.1038/s41587-021-01145-6) mentions following statistics about the knowledge graph contents: 
- Close to 20 million nodes having 36 node types
- More than 200 million edges having 47 edge types

### a) Number of nodes and edges

In [ ]:
%%time

num_nodes = len(df_nodes)
num_edges = len(df_edges)

print(f"{num_nodes:,} nodes")
print(f"{num_edges:,} edges")
print()

Interpretation:
- Inspecting the raw data resulted in **14,543,042 nodes**, while the publication mentioned **close to 20,000,000 nodes**.
- Inspecting the raw data resulted in **188,266,233 edges**, while the publication mentioned **more than 200,000,000 edges**.
- Both **differences** could have a variety of reasons, e.g. the authors may have changed the process that generates the data from Version 1 to Version 3 after the publication, there may be an error in the reported numbers, or the data extraction via Neo4j import and export inside a Docker container may have a defect.

### b) Types of nodes and edges

In [ ]:
%%time

nt_column = "type"
nt_counts = df_nodes.groupby(nt_column).size().compute()
nt_counts = nt_counts.sort_values(ascending=False)

print(len(nt_counts), "node types, sorted by their frequency:")
for key, val in nt_counts.items():
    print(f"- {key}: {val}")
print()

In [ ]:
%%time

et_column = "type"
et_counts = df_edges.groupby(et_column).size().compute()
et_counts = et_counts.sort_values(ascending=False)

print(len(et_counts), "edge types, sorted by their frequency:")
for key, val in et_counts.items():
    print(f"- {key}: {val}")
print()

In [ ]:
# Correctness checks

# 1) Do the counts of different node types add up to the total number of nodes?
sum_node_types = nt_counts.sum()
assert sum_node_types == num_nodes, f"Node counts differ: {sum_node_types} != {num_nodes}"
print(f"{sum_node_types:,} = {num_nodes:,} nodes")

# 2) Do the counts of different edge types add up to the total number of edges?
sum_edge_types = et_counts.sum()
assert sum_edge_types == num_edges, f"Edge counts differ: {sum_edge_types} != {num_edges}"
print(f"{sum_edge_types:,} = {num_edges:,} edges")

Interpretation:
- Inspecting the raw data resulted in **32 node types**, while the publication mentions **36 node types**, which is 4 more.
- Inspecting the raw data resulted in **39 edge types**, while the publication mentions **47 edge types**, which is 8 more.
- Both **differences** could have a variety of reasons, probably the same as the differences observed in node and edge counts.
- Looking at **relative frequencies of node and edge types** suggests that the dataset is rather unbalanced.
  - The most frequent node type is "Known_variant" with 10,630,108 instances, while the least frequent node type is "User" with only 2 instances, a difference of 7 orders of magnitude.
  - The most frequent edge type is "MENTIONED_IN_PUBLICATION" with 111,109,238 instances, while the least frequent edge types only have 7 instance each, a difference of 8 orders of magnitude.

### c) Example entries

This section prints some example entries of the raw data. It gives an impression of the format chosen by the authors, which differs greatly between projects due to a lack of a broadly accepted standard for biomedical knowledge graphs.

In [ ]:
def report_first_n_items(data, n):
    return data.head(n)

In [ ]:
def report_last_n_items(data, n):
    return data.tail(n)

#### Nodes together with node annotations

In [ ]:
report_first_n_items(df_nodes, 2)

In [ ]:
report_last_n_items(df_nodes, 2)

#### Edges together with edge annotations

In [ ]:
report_first_n_items(df_edges, 2)

In [ ]:
report_last_n_items(df_edges, 2)

## 6. Schema detection

This section analyzes the structure of the knowledge graph by determining which types of nodes are connected by which types of edges. To construct this overview, it is necessary to iterate over the entire data once. The result is a condensed representation of all entities and relations, which is known as [data model](https://neo4j.com/docs/getting-started/data-modeling/guide-data-modeling/#whiteboard-friendly) or [schema](https://memgraph.com/docs/fundamentals/graph-modeling#designing-a-graph-database-schema) in the context of graph databases.

In [ ]:
node_type_to_color = {
    "Metabolite": "green",

    "Gene": "blue",
    "Transcript": "blue",
    "Protein": "blue",
    "Modified_protein": "blue",
    "Peptide": "blue",

    "Disease": "red",
    "Pathway": "red",
    "Biological_process": "red",
}

In [ ]:
%%time

node_id_to_type = {row.id: row.type for row in df_nodes.itertuples()}

In [ ]:
%%time

unique_triples = set()
for row in df_edges.itertuples():
    s = node_id_to_type[row.source_id]
    p = row.type
    o = node_id_to_type[row.target_id]
    triple = (s, p, o)
    unique_triples.add(triple)

In [ ]:
gs = ig.Graph(directed=True)
unique_nodes = set()
for s, p, o in unique_triples:
    for node in (s, o):
        if node not in unique_nodes:
            unique_nodes.add(node)
            
            node_size = int(nt_counts[node])
            node_color = node_type_to_color.get(node, '')
            node_hover = f"{node}\n\n{nt_counts[node]} nodes of this type are contained in the knowledge graph."
            gs.add_vertex(node, size=node_size, color=node_color, label_color=node_color, hover=node_hover)

    edge_size = int(et_counts[p])
    edge_color = node_type_to_color.get(s, '')
    edge_hover = f"{p}\n\n{et_counts[p]} edges of this type are contained in the knowledge graph."
    gs.add_edge(s, o, size=edge_size, color=edge_color, hover=edge_hover, label=p, label_color="gray", label_size=5)

gs.vcount(), gs.ecount()

In [ ]:
fig = gv.d3(
    gs,
    show_node_label=True,
    node_label_data_source="name",

    show_edge_label=True,
    edge_label_data_source="label",
    edge_curvature=0.2,

    use_node_size_normalization=True,
    node_size_normalization_min=10,
    node_size_normalization_max=50,
    node_drag_fix=True,
    node_hover_neighborhood=True,
    
    use_edge_size_normalization=True,
    edge_size_normalization_max=3,

    many_body_force_strength=-3000,
    zoom_factor=0.3,
)
fig

In [ ]:
# Export the schema visualization
schema_filepath = os.path.join(results_dir, f"{project_name}_schema.html")
fig.export_html(schema_filepath, overwrite=True)

Interpretation:
- Each node in the schema corresponds to one of the 32 node types in the data.
  - *Node size* represents the number of instances, i.e. how often that node type is present in the knowledge graph. The exact number can also be seen when hovering over a node. The large differences indicate again that the dataset is rather unbalanced.
  - *Node color* represents particular node types. The coloring scheme is based on a deliberately simple RGB palette with the same meaning across multiple notebooks to enable some visual comparison. The idea behind it is to highlight an interplay of certain entities, namely that drugs (or small molecules in general) can bind to proteins (or gene products in general) and thereby alter diseases (or involved pathways).
    - *green* = drugs & other small molecules (e.g. toxins)
    - *blue* = genes & gene products (e.g. proteins or RNAs)
    - *red* = diseases & related concepts (e.g. pathways)
    - *black* = all other types of entities
- Each edge in the schema stands for one of the 39 edge types in the data. It is possible that the same edge type appears between different nodes.
  - *Edge size* represents the number of instances, i.e. how often that edge type is present in the knowledge graph.
  - *Edge color* is identical to the color of the source node, again to highlight the interplay between drugs, targets and diseases.

## 6. Knowledge graph reconstruction

Usually, this section first converts the raw data to an intermediate format used in several notebooks, and then reconstructs the knowledge graph from the standardized data with shared code. However, in the case of CKG, the data conversion was already done in section 3 that extracted the nodes and edges from the provided Neo4j database. Further, only a part of the knowledge graph will be reconstructed for memory reasons.
- The intermediate form of the data is created as two simple Python lists, one for nodes and the other for edges, which can be exported to two CSV files.
- The knowledge graph is built as a graph object from the Python package [igraph](https://igraph.org/python), which can be exported to a [GraphML](https://en.wikipedia.org/wiki/GraphML) file.

### a) Convert the data into a standardized format

Transform the raw data to an standardized format that is compatible with most biomedical knowledge graphs in order to enable shared downstream processing:
- Each node is represented by three items: `id (str), type (str), properties (dict)`
- Each edge is represented by four items: `source_id (str), target_id (str), type(str), properties (dict)`

This format was initially inspired by a straightforward way in which the content of a Neo4j graph database can be exported to two CSV files, one for all nodes and the other for all edges. This is an effect of the [property graph model](https://neo4j.com/docs/getting-started/appendix/graphdb-concepts/) used in Neo4j and many other graph databases, which also appears to be general enough to fully capture the majority of biomedical knowledge graphs described in scientific literature, despite the large variety of formats they are shared in.

A second motivation was that each line represents a single node or edge, and that no entry is connected to any sections at other locations, such as property descriptions at the beginning of a GraphML file. This structural simplicity makes it very easy to load just a subset of nodes and edges by picking a subset of lines, or to skip the loading of properties if they not required for a task simply by ignoring a single column.

Finally, this format also allows to load the data directly into popular SQL databases like [SQLite](https://www.sqlite.org), [MySQL](https://www.mysql.com) or [PostgreSQL](https://www.postgresql.org/) with built-in CSV functions ([CSV in SQLite](https://www.sqlite.org/draft/cli.html#importing_files_as_csv_or_other_formats), [CSV in MySQL](https://dev.mysql.com/doc/refman/en/loading-tables.html), [CSV in PostgreSQL](https://www.postgresql.org/docs/current/sql-copy.html)). Further, the JSON string in the property column can be accessed directly by built-in JSON functions ([JSON in SQLite](https://www.sqlite.org/json1.html), [JSON in MySQL](https://dev.mysql.com/doc/refman/en/json-function-reference.html), [JSON in PostgreSQL](https://www.postgresql.org/docs/current/functions-json.html)), which enables sophisticated queries that access or modify specific properties within the JSON data.

#### Subset of node ids

**Caution: Only a part of the knowledge graph is reconstructed here**, because the entire dataset is too large to load it completely into a igraph graph object in memory on an average laptop. To do so, first a subset of interesting nodes is identified (associated with the Imatinib/CML story that follows later), then all edges are collected that contain these nodes as source or target, and finally all nodes are collected that appear on either side of these edges.

In [ ]:
%%time

substrings = ["bcr", "abl1", "imatinib", "chronic myeloid leukemia"]
column = "properties"
mask = df_nodes[column].str.contains(substrings[0], case=False, na=False)
for substring in substrings[1:]:
    mask |= df_nodes[column].str.contains(substring, case=False, na=False)

df_nodes_filtered = df_nodes[mask].compute()
node_ids = set(df_nodes_filtered["id"])

#### Edges

In [ ]:
%%time

mask = df_edges["source_id"].isin(node_ids) | df_edges["target_id"].isin(node_ids)
edges = df_edges[mask].compute().values.tolist()
edges = [(sid, tid, etype, json.loads(eproperties)) for sid, tid, etype, eproperties in edges]

#### Nodes

In [ ]:
source_ids = [sid for sid, tid, etype, eproperties in edges]
target_ids = [tid for sid, tid, etype, eproperties in edges]
node_ids_in_edges = set(source_ids + target_ids)

In [ ]:
%%time

mask = df_nodes["id"].isin(node_ids_in_edges)
nodes = df_nodes[mask].compute().values.tolist()
nodes = [(nid, ntype, json.loads(nproperties)) for nid, ntype, nproperties in nodes]

### b) Export the standardized data to two CSV files

Both the `id` and `type` items are simple strings, while the `properties` item is collection of key-value pairs represented by a Python dictionary that can be converted to a single JSON string, which the export function does internally. This means each node is fully represented by three strings, and each edge by four strings due to having a source id and target id.

In [ ]:
nodes_csv_filepath = shared_bmkg.export_nodes_as_csv(nodes, results_dir, project_name + "_subset")

In [ ]:
edges_csv_filepath = shared_bmkg.export_edges_as_csv(edges, results_dir, project_name + "_subset")

### c) Use the standardized data to build a graph

Reconstruct a part of the knowledge graph in form of a [Graph object](https://igraph.org/python/doc/api/igraph.Graph.html) from the package [igraph](https://igraph.org/python). This kind of graph object allows to have directed multi-edges, i.e. an edge has a source and a target node, and two nodes can be connected by more than one edge. It also allows to have node and edge properties. These features are necessary and sufficient to represent almost any biomedical knowledge graph found in academic literature.

In [ ]:
g = shared_bmkg.create_graph(nodes, edges)
shared_bmkg.report_graph_stats(g)

In [ ]:
# Correctness checks

# 1) Does the reconstructed graph contain the same number of nodes as the raw data?
num_nodes_in_data = len(nodes)
num_nodes_in_graph = g.vcount()
assert num_nodes_in_graph == num_nodes_in_data, f"Node counts differ: {num_nodes_in_graph} != {num_nodes_in_data}"
print(f"{num_nodes_in_graph:,} = {num_nodes_in_data:,}")

# 2) Does the reconstructed graph contain the same number of (unique) edges as the raw data?
num_edges_in_data = len(edges)
num_edges_in_graph = g.ecount()
assert num_edges_in_graph == num_edges_in_data, f"Edge counts differ: {num_edges_in_graph} != {num_edges_in_data}"
print(f"{num_edges_in_graph:,} = {num_edges_in_data:,}")

Select the largest connected component of the graph in order to get rid of some small disconnected subgraphs

In [ ]:
g = g.connected_components(mode="weak").giant()
shared_bmkg.report_graph_stats(g)

### d) Export the graph to a GraphML file

Export the graph with all nodes, edges and properties as a single [GraphML](https://en.wikipedia.org/wiki/GraphML) file.

In [ ]:
%%time

g_graphml_filepath = shared_bmkg.export_graph_as_graphml(g, results_dir, project_name)

## 7. Subgraph exploration

This section explores small subgraphs of the knowledge graph in two ways: first by inspecting the direct neighborhood of a selected node, and second by finding shortest paths between two chosen nodes.

As a simple case study, the goal is to identify some nodes in the knowledge graph that are associated with the success story of the drug Imatinib, which was one of the first [targeted therapies](https://en.wikipedia.org/wiki/Targeted_therapy) against cancer. Detailed background information can for example be found in an article by the [National Cancer Institute](https://www.cancer.gov/research/progress/discovery/gleevec) and in a [talk by Brian Druker](https://www.ibiology.org/human-disease/imatinib-paradigm-targeted-cancer-therapies) who played a major role in the development of this paradigm-changing drug. To give a simplified summary, following biological entities and relationships are involved:

- Mutation: In a bone marrow stem cell, a translocation event between chromosome 9 and 22 leads to what has been called the [Philadelphia chromosome](https://en.wikipedia.org/wiki/Philadelphia_chromosome), which can be seen under a microscope and got named after the city it originally got discovered in.
- Gene: It turned out that this particular rearrangement of DNA fuses the [BCR](https://en.wikipedia.org/wiki/BCR_(gene)) gene on chromosome 22 to the [ABL1](https://en.wikipedia.org/wiki/ABL_(gene)) gene on chromosome 9, resulting in a new fusion gene known as BCR-ABL1.
- Disease: BCR-ABL1 acts as an oncogene, because it expresses a protein that is a defective [tyrosine kinase](https://en.wikipedia.org/wiki/Tyrosine_kinase) in a permanent "on" state, which leads to uncontrolled growth of certain white blood cells and their precursors, thereby driving the disease [Chronic Myelogenous Leukemia (CML)](https://en.wikipedia.org/wiki/Chronic_myelogenous_leukemia).
- Drug: [Imatinib (Gleevec)](https://en.wikipedia.org/wiki/Imatinib) was the first demonstration that a potent and selective [Bcr-Abl tyrosine-kinase inhibitor (TKI)](https://en.wikipedia.org/wiki/Bcr-Abl_tyrosine-kinase_inhibitor) is possible and that such a targeted inhibition of an oncoprotein halts the uncontrolled growth of leukemia cells with BCR-ABL1, while having significantly less effect on other cells in the body compared to conventional chemotherapies used in cancer. This revolutionized the treatment of CML and drastically improved the five-year survival rate of patients from less than 20% to over 90%, as well as their quality of life.

In reality the story is a bit more complex, for example because there are other genes involved in disease progression, there are many closely related forms of leukemia, BCR-ABL1 also plays a role in other forms of cancer, there are several drugs available as treatment options today, all of them bind to more than one target and with different affinities, and their individual binding profiles are relevant to their particular therapeutic effects. Inspecting the knowledge graph will focus on highlighting some entities of the simplified story, but the surrounding elements will also indicate some of the complexity encountered in reality. Some simple forms of reasoning on the knowledge graph will demonstrate its potential for discovering new patterns and hypotheses.

### a) Search for interesting nodes

In [ ]:
# Drug: Imatinib
shared_bmkg.list_nodes_matching_substring(g, "imatinib", "_name")

In [ ]:
# Gene: ABL1
shared_bmkg.list_nodes_matching_substring(g, "abl1", "_name")  # Key "name" had to be replaced with "_name" to prevent an issue with igraph

In [ ]:
# Disease: Myeloid Leukemia - to find Chronic Myeloid Leukemia (CML)
shared_bmkg.list_nodes_matching_substring(g, "myeloid leukemia", "_name")

### b) Explore the neighborhood of a chosen node

In [ ]:
# Neighborhood of drug Imatinib
source = "14709403"
subgraph = shared_bmkg.get_egocentric_subgraph(g, source)

# Export
filename = f"{project_name}_neighbors_imatinib"
shared_bmkg.export_graph_as_graphml(subgraph, results_dir, filename)
shared_bmkg.export_nodes_as_csv(nodes, results_dir, filename, subgraph)
shared_bmkg.export_edges_as_csv(edges, results_dir, filename, subgraph)

# Report
shared_bmkg.report_graph_stats(subgraph)
shared_bmkg.visualize_graph(subgraph, node_type_to_color, source=source)

Interpretation:
- The drug Imatinib (green node at the center) is connected only to one other node, a pathway named "Imatinib Inhibition of BCR-ABL".
- **Caution**: This small neighborhood is most likely a result of the filtering step that had to be performed in order to get to a reasonably sized graph that fits into memory. It does not reflect all information about Imatinib in CKG!

In [ ]:
# Neighborhood of protein ABL1
source = "415687"
subgraph = shared_bmkg.get_egocentric_subgraph(g, source)

# Export
filename = f"{project_name}_neighbors_abl1"
shared_bmkg.export_graph_as_graphml(subgraph, results_dir, filename)
shared_bmkg.export_nodes_as_csv(nodes, results_dir, filename, subgraph)
shared_bmkg.export_edges_as_csv(edges, results_dir, filename, subgraph)

# Report
shared_bmkg.report_graph_stats(subgraph)
#shared_bmkg.visualize_graph(subgraph, node_type_to_color, source=source)

Interpretation:
- CKG contains too many nodes connected to the protein ABL1 for plotting it and performing a visual analysis.
- **Caution**: This might not even be the entire neighborhood due to the filtering step that had to be performed in order to get to a reasonably sized graph that fits into memory.

In [ ]:
# Neighborhood of disease CML
source = "10265"
subgraph = shared_bmkg.get_egocentric_subgraph(g, source)

# Export
filename = f"{project_name}_neighbors_cml"
shared_bmkg.export_graph_as_graphml(subgraph, results_dir, filename)
shared_bmkg.export_nodes_as_csv(nodes, results_dir, filename, subgraph)
shared_bmkg.export_edges_as_csv(edges, results_dir, filename, subgraph)

# Report
shared_bmkg.report_graph_stats(subgraph)

Interpretation:
- CKG contains too many nodes connected to the disease CML for plotting it and performing a visual analysis.
- **Caution**: This might not be the entire neighborhood due to the filtering step that had to be performed in order to get to a reasonably sized graph that fits into memory.

### c) Find shortest paths between two chosen nodes

In [ ]:
# Paths from transcript "tyrosine-protein kinase ABL1 isoform a" to disease AML
source = "270788"
target = "10440"
subgraph = shared_bmkg.get_paths_subgraph(g, source, target)

# Report
shared_bmkg.report_graph_stats(subgraph)
shared_bmkg.visualize_graph(subgraph, node_type_to_color, source, target)

Interpretation:
- The transcript "tyrosine-protein kinase ABL1 isoform a" (blue node on the left) leads via a single path to the disease AML (red node on the right). It is connected via a "TRANSLATED_INTO" relation to the protein ABL1, which in turn is linked by a "ASSOCIATED_WITH" relation to the disease AML.
- **Caution**: These might not be all paths due to the filtering step that had to be performed in order to get to a reasonably sized graph that fits into memory.